In [ ]:
import numpy as np
import pandas as pd
import os
import sklearn.metrics as skm
import re
import numpy as np
from string import digits

pd.set_option('display.max_rows', None)

In [ ]:
# Panel panel_a1
panel_a1 = [
    "inflation expectation", "interest rate", "bank rate", "fund rate", "price",
    "economic activity", "inflation", "employment"
]

# Panel panel_a2
panel_a2 = [
    "anchor", "cut", "subdue", "decline", "decrease", "reduce", "low",
    "drop", "fall", "fell", "decelerate", "slow", "pause", "pausing",
    "stable", "non-accelerating", "downward", "tighten"
]
# Panel panel_b1
panel_b1 = [
    "unemployment", "growth", "exchange rate", "productivity", "deficit",
    "demand", "job market", "monetary policy"
]

# Panel panel_b2
panel_b2 = [
    "ease", "easing", "rise", "rising", "increase", "expand", "improve",
    "strong", "upward", "raise", "high", "rapid"
]

panel_c = ["weren't", "were not", "wasn't", "was not", 'did not', "didn't", "do not", "don't", 'will not', "won't"]


def rule_model(df):
    articles = df.tolist()
    pred = []
    for s in articles:
        label = 0
        if (any(word in s.lower() for word in panel_a1) and any(word in s.lower() for word in panel_a2)) or \
                (any(word in s.lower() for word in panel_b1) and any(word in s.lower() for word in panel_b2)):
            label = 0
        elif (any(word in s.lower() for word in panel_a1) and any(word in s.lower() for word in panel_b2)) or \
                (any(word in s.lower() for word in panel_b1) and any(word in s.lower() for word in panel_a2)):
            label = 1
        else:
            label = 2
        if label != 2 and (any(word in s.lower() for word in panel_c)):
            pred.append(1 - label)  # turn 0 to 1, and 1 to 0
        else:
            pred.append(label)

    return pred

In [ ]:
test_articles = pd.read_excel('OpenAI_Scored_Articles_3000.xlsx')

# Use random sample of 1000 articles
test_articles = test_articles.sample(1000, random_state=13)
test_articles.dropna(inplace=True)
test_articles['OpenAI_Score'] = test_articles['OpenAI_Score'].map({0: 'Neutral', -1: 'Dovish', 1: 'Hawkish'})
test_articles.rename(columns={'Manual.summary': 'article', 'OpenAI_Score':'label'}, inplace=True)
print(test_articles.shape)
test_articles.head()



In [ ]:
test_articles['label'].value_counts()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from tqdm import tqdm

# Function to perform the classification and calculate F1 score
def classify_and_evaluate(seed):
    # Read the data
    test_articles = pd.read_excel('OpenAI_Scored_Articles_3000.xlsx')

    # Use random sample of 1000 articles with specified seed
    test_articles = test_articles.sample(1000, random_state=seed)
    test_articles.dropna(inplace=True)
    test_articles['OpenAI_Score'] = test_articles['OpenAI_Score'].map({0: 'Neutral', -1: 'Dovish', 1: 'Hawkish'})
    test_articles.rename(columns={'Manual.summary': 'article', 'OpenAI_Score':'label'}, inplace=True)

    # Apply rule-based model to predict labels
    test_articles['dict_label'] = rule_model(test_articles['article'])
    test_articles['dict_label'] = test_articles['dict_label'].map({0: 'Dovish', 1: 'Hawkish', 2: 'Neutral'})

    # Compute accuracy
    test_articles['Corrects'] = np.where(test_articles['dict_label'] == test_articles['label'], 1, 0)
    accuracy = test_articles['Corrects'].sum() / len(test_articles)

    # Encode labels for F1 score calculation
    def encode_labels(labels):
        label_map = {"Hawkish": 0, "Dovish": 1, "Neutral": 2}
        return [label_map[label] for label in labels]

    test_articles['label_encoded'] = encode_labels(test_articles['label'])
    test_articles['dict_label_encoded'] = encode_labels(test_articles['dict_label'])

    # Calculate F1 score
    f1 = f1_score(test_articles['label_encoded'], test_articles['dict_label_encoded'], average='weighted')

    return accuracy, f1

# Initialize variables to store best seed and corresponding f1 score
best_seed = None
best_f1_score = -1

# Loop over different random seeds and compute F1 scores
for seed in tqdm(range(1000)):
    accuracy, f1 = classify_and_evaluate(seed)
    if f1 > best_f1_score:
        best_f1_score = f1
        best_seed = seed

print(f"Best seed: {best_seed}, Best F1 score: {best_f1_score:.4f}")

In [ ]:
test_articles['label'].value_counts()

In [ ]:
test_articles['dict_label'] = rule_model(test_articles['article'])
# Apply dictionary 0 is dovish, 1 is hawkish and 2 is neutral
test_articles['dict_label'] = test_articles['dict_label'].map({0: 'Dovish', 1: 'Hawkish', 2: 'Neutral'})
print(test_articles)

In [ ]:
from sklearn.metrics import f1_score

# Compare dict_label with Classification Rui to see how many are the same
test_articles['Corrects'] = np.where(test_articles['dict_label'] == test_articles['label'], 1, 0)
print(test_articles['Corrects'].sum())

In [ ]:
# Example conversion function
def encode_labels(labels):
    label_map = {"Hawkish": 1, "Dovish": 0, "Neutral": 2}
    return [label_map[label] for label in labels]

# Example usage
test_articles['label_encoded'] = encode_labels(test_articles['label'])
test_articles['dict_label_encoded'] = encode_labels(test_articles['dict_label'])

f1 = f1_score(test_articles['label_encoded'], test_articles['dict_label_encoded'], average='weighted')
print(f"F1 score: {f1:.4f}")

In [ ]:
test_articles.head()

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

actual_labels = test_articles['label_encoded']
predicted_labels = test_articles['dict_label_encoded']

# Create confusion matrix
cm = confusion_matrix(actual_labels, predicted_labels, labels=[0, 1, 2])

# Print classification report
print(classification_report(actual_labels, predicted_labels))

# Create a test_dfframe for better visualization of the confusion matrix
cm_df = pd.DataFrame(cm, index=['Actual_Dovish', 'Actual_Hawkish', 'Actual_Neutral'],
                     columns=['Predicted_Dovish', 'Predicted_Hawkish', 'Predicted_Neutral'])

print("Confusion Matrix:")
print(cm_df)

# Extracting TP, FP, TN, FN for each class
# For 'Dovish'
tp_dovish = cm[0, 0]
fp_dovish = cm[1:, 0].sum()
fn_dovish = cm[0, 1:].sum()
tn_dovish = cm[1:, 1:].sum()

# For 'Hawkish'
tp_hawkish = cm[1, 1]
fp_hawkish = cm[[0, 2], 1].sum()
fn_hawkish = cm[1, [0, 2]].sum()
tn_hawkish = cm[[0, 2], :][:, [0, 2]].sum()

# For 'Neutral'
tp_neutral = cm[2, 2]
fp_neutral = cm[:2, 2].sum()
fn_neutral = cm[2, :2].sum()
tn_neutral = cm[:2, :2].sum()

print(f"Dovish - TP: {tp_dovish}, FP: {fp_dovish}, FN: {fn_dovish}, TN: {tn_dovish}")
print(f"Hawkish - TP: {tp_hawkish}, FP: {fp_hawkish}, FN: {fn_hawkish}, TN: {tn_hawkish}")
print(f"Neutral - TP: {tp_neutral}, FP: {fp_neutral}, FN: {fn_neutral}, TN: {tn_neutral}")